## Optionals. Improving Null Safety

### Optionals

Optionals are not functional interfaces, instead it's a nifty utility to prevent <code>NullPointerException</code>

Optional is a simple container for a value which may be null or non-null. Think of a method which may return a non-null result but sometimes return nothing. Instead of returning null you return an <code>Optional</code> in Java 8.

In [1]:
Optional<String> optional = Optional.of("bam");

In [2]:
optional.isPresent();

true

In [3]:
optional.get();

bam

In [4]:
optional.orElse("fallback");

bam

In [5]:
optional.ifPresent(s -> System.out.println(s.charAt(0)));

b


### Improving Null Safety

We can utilize the <code>Optional</code> type of Java 8 to prevent <code>null</code> checks. 

In [15]:
class Inner {
    private String foo;
    
    Inner(String foo) {
        this.foo = foo;
    }
    public String getFoo() {
        return foo;
    }
}

class Nested {
    private Inner inner;
    
    Nested(Inner inner) {
        this.inner = inner;
    }
    
    public Inner getInner() {
        return inner;
    }
}

class Outer {
    private Nested nested;
    
    Outer() {
        
    }
    
    Outer(Nested nested) {
        this.nested = nested;
    }
    public Nested getNested() {
        return nested;
    }
}

Resolving a deep nested path in this structure can be kinda awkward. We have to write a bunch of null checks to make sure not to raise a <code>NullPointerException</code>:

In [16]:
Outer outer = new Outer();
if (outer != null && outer.getNested() != null && 
    outer.getNested().getInner() != null) {
    System.out.println(outer.getNested().getInner().getFoo());
}

We can get rid of all those null checks by utilizing the Java 8 <code>Optional</code> type. The method map accepts a lambda expression of type <code>Function</code> and automatically wraps each function result into an <code>Optional</code>. That enables us to pipe multiple <code>map</code> operations in a row. Null checks are automatically handled under the hood:

In [10]:
Optional.of(new Outer())
    .map(Outer::getNested)
    .map(Nested::getInner)
    .map(Inner::getFoo)
    .ifPresent(System.out::println);

In [18]:
Outer outer = new Outer(new Nested(new Inner("foo param")));
Optional.of(outer)
    .map(Outer::getNested)
    .map(Nested::getInner)
    .map(Inner::getFoo)
    .ifPresent(System.out::println);

foo param


An alternative way to achieve the same behavior is by utilizing a supplier function to resolve the nested path:

In [23]:
import java.util.function.Supplier;

class Util {
    static <T> Optional<T> resolve(Supplier<T> resolver) {
        try {
            T result = resolver.get();
            return Optional.ofNullable(result);
        }
        catch (NullPointerException e) {
            return Optional.empty();
        }
    }
}

In [ ]:
Outer obj = new Outer();
Util.resolve(() -> obj.getNested().getInner().getFoo())
    .ifPresent(System.out::println);

In [25]:
Outer obj = new Outer(new Nested(new Inner("foo param")));
Util.resolve(() -> obj.getNested().getInner().getFoo())
    .ifPresent(System.out::println);

foo param


Calling <code>obj.getNested().getInner().getFoo())</code> might throw a <code>NullPointerException</code>. In this case the exception will be caught and the method returns <code>Optional.empty()</code>.